In [1]:
import os
from os.path import exists

import pandas as pd
from shapely import geometry
import numpy as np
from glob import glob 
import rasterio
import cv2
from tenacity import retry, stop_after_attempt
from tqdm import tqdm 
import matplotlib.pyplot as plt
from multiprocessing import Pool
import urllib
import time 
from matplotlib import pyplot as plt 
import fiona
output_path = "../data/test_data/"

In [2]:
from ultralyticsplus import YOLO, render_result

model_name = 'keremberke/yolov8s-building-segmentation'
# load model
model = YOLO(model_name)

# set model parameters
model.overrides['conf'] = 0.40  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

#### Function for Sattlite Image Generation

In [3]:
@retry(stop=stop_after_attempt(3))
def extractSatelliteImages(minX, minY, maxX, maxY, height='512', width='512'):
    url = f"http://wms3.mapsavvy.com/WMSService.svc/db45ac1c32ac4e9caa5ecc3473998c81/WMSLatLon?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&LAYERS=Aerial&SRS=EPSG:4326&CRS=EPSG:4326&BBOX={minX},{minY},{maxX},{maxY}&WIDTH={width}&HEIGHT={height}&STYLES=&TRANSPARENT=false&FORMAT=image/png"
    req = urllib.request.Request(url)
    resp = urllib.request.urlopen(req)
    arr = np.asarray(bytearray(resp.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#### Functions for Pix2Coordinates

In [4]:
from shapely.geometry import LineString, Polygon
from PIL import Image
from rasterio import features
from collections import defaultdict
from shapely import geometry
# from centerline.geometry import Centerline
from shapely.ops import linemerge
from affine import Affine


def extract_leads(
    image: Image, simplify_dist: float = 0.00005, extent: tuple = None) -> defaultdict:
    transform = (
        rasterio.transform.from_bounds(*extent, width=image.shape[0], height=image.shape[1])
        if extent
        else Affine.identity()
    )
    shapes = features.shapes(np.array(image), transform=transform)
    leads = defaultdict(list)
    
    for poly_geojson, value in shapes:
        if value != 0:
            poly = geometry.Polygon(poly_geojson["coordinates"][0])
            return poly 

def convertToTiff(img, minlat, minlong, maxlat, maxlong, height=512, width=512, imagename='sample_image',
                    path='output', crs="epsg:4326",chanenls=3,prefix=1):
    if not os.path.isdir(path):
        os.makedirs(path)

    filename = imagename + '.tiff'
    if os.path.isfile(filename):
        return 
    transform = rasterio.transform.from_bounds(minlong, minlat, maxlong, maxlat, width, height)
    with rasterio.open(os.path.join(path, filename), 'w', driver='GTiff', dtype=rasterio.uint8, count=3,
                        width=width, height=height, transform=transform, crs=crs) as dst:
        for index in range(chanenls):
            dst.write(img[:, :, index], indexes=index + 1)

In [15]:
import geopandas as gpd 
import mercantile

def extract_bfps_for_tile(tile_path,tile_id, crs='4326',out_path='output_data',zoom=16,img_height=1024,img_width=1024,save_img = False):
    gdf = gpd.read_file(tile_path)
    quad_key = os.path.basename(tile_path).split('.')[0]
    
    outer_bounds = gdf.total_bounds
    spartial_index = gdf.sindex
    tiles = mercantile.tiles(outer_bounds[0], outer_bounds[1], outer_bounds[2], outer_bounds[3], zooms=16)
    
    polygons = []
    
    for i, tile in tqdm(enumerate(tiles)):
        val = mercantile.bounds(tile)
        img = extractSatelliteImages(val.west, val.south, val.east, val.north,
                                            height=str(img_height),
                                            width=str(img_width),
                                            )
        resizeImg = cv2.resize(img,(650,650)) # input size for model 
        results = model.predict(resizeImg)
        if results[0].masks:
            if save_img:
                convertToTiff(img, val.south,val.west,val.north,val.east,
                            height=img_height,
                            width=img_width,
                            path=os.path.join(out_path,'image'),
                            imagename='{}_{}_{}'.format(tile_id,zoom,i)
                            )
            for mask_img in results[0].masks.masks:
                output_mask_np = mask_img.detach().cpu().numpy()
                output_mask_np = cv2.resize(output_mask_np,(img_height,img_width))
                output_mask_np[np.where(output_mask_np>0)] = 1
                geom = extract_leads(image = output_mask_np,extent=(val[0], val[1], val[2], val[3]))
                if geom.geom_type == "Polygon":
                    polygons.append(geom)       
            
    df = pd.DataFrame({'quad_key':'france_city','geometry':polygons})
    gdf = gpd.GeoDataFrame(df,geometry='geometry',crs='epsg:4326')
    with fiona.Env(OSR_WKT_FORMAT="WKT2_2018"):
        gdf.to_file(os.path.join(output_path,f"{quad_key}_polygons.geojson"),driver="GeoJSON")
    # inference_gdf.to_file(f"{output_path}/{quad_key}_Polygons.geojson", driver="GeoJSON",mode='a')
    return 0 


In [6]:
# this is the name of the geography you want to retrieve. update to meet your needs
location = 'France'

dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
location_links = dataset_links[dataset_links.Location == location]
print("Found {} links for {}".format(len(location_links),location))

Found 236 links for France


### Test the Images with YOLOv8 Model 

In [30]:
file_path = "../data/london_city.shp"
df = gpd.read_file(file_path)

quad_key = os.path.basename(file_path).split('.')[0]
print(quad_key)
df.head()

london_city


,id,geometry
0,0,"POLYGON ((-0.43090 51.63266, 0.23599 51.63586,..."
1,0,"POLYGON ((-1.58131 49.20523, -1.57761 49.20504..."


In [31]:
tile_data = glob(os.path.join(output_path+'/*.geojson'))

# for tile_file in tile_data:
tile_id = os.path.basename(file_path).split('.')[0]
poly  = extract_bfps_for_tile(tile_path =file_path,
                                tile_id='london_city',
                                out_path=output_path)


0: 640x640 (no detections), 120.2ms
Speed: 0.4ms preprocess, 120.2ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)
271it [30:19,  5.25s/it]
0: 640x640 (no detections), 112.3ms
Speed: 0.5ms preprocess, 112.3ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)
272it [30:26,  5.84s/it]
0: 640x640 (no detections), 125.5ms
Speed: 0.6ms preprocess, 125.5ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)
273it [30:31,  5.50s/it]
0: 640x640 (no detections), 128.0ms
Speed: 0.6ms preprocess, 128.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
274it [30:42,  7.00s/it]
0: 640x640 (no detections), 123.0ms
Speed: 0.3ms preprocess, 123.0ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)
275it [30:46,  6.20s/it]
0: 640x640 (no detections), 115.3ms
Speed: 0.5ms preprocess, 115.3ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)
276it [30:50,  5.63s/it]
0: 640x640 (no detections), 130.4ms
Speed: 0.

In [25]:
df = pd.DataFrame({'quad_key':'london_city','geometry':poly})
gdf = gpd.GeoDataFrame(df,geometry='geometry',crs='epsg:4326')
with fiona.Env(OSR_WKT_FORMAT="WKT2_2018"):
    gdf.to_file(os.path.join(output_path,f"{quad_key}_polygons.geojson"),driver="GeoJSON")

In [ ]:
from plantcv import plantcv as pcv
from shapely.geometry import LineString, Polygon
import traceback

def convertor(value, src_min, src_max, dest_min, dest_max):
    src_range = src_max - src_min
    dest_range = dest_max - dest_min
    value_scaled = float(value - src_min) / float(src_range)
    return dest_min + (value_scaled * dest_range)

def coord2pix(xycoord, bbox, height, width):
    return [convertor(xycoord[0], bbox['minx'], bbox['maxx'], 0, width), convertor(xycoord[1], bbox['miny'], bbox['maxy'] , height, 0)]


def pix2coord(xypixel, bbox, height, width):
    return [convertor(xypixel[0], 0, width, bbox['minx'], bbox['maxx']), convertor(xypixel[1], height, 0, bbox['miny'], bbox['maxy'])]

def mercantilebbox2bbox(bbox_obj):
    return {'maxx': bbox_obj.east, 'minx': bbox_obj.west, 'maxy': bbox_obj.north, 'miny': bbox_obj.south}

def convert_pix2coord(geom, quad_key):
    try:
        tile = mercantile.quadkey_to_tile(quad_key)
        bbox_obj = mercantile.bounds(tile)
        bbox = mercantilebbox2bbox(bbox_obj)
        print(bbox)
        if geom.geom_type == "LineString":
            geom_coords = list(geom.coords)
            geom = LineString([pix2coord(xypixels, bbox, IMG_HEIGHT, IMG_WIDTH) for xypixels in geom_coords])
        elif geom.geom_type == "Polygon":
            geom_coords = list(geom.exterior.coords)
            print("Polygon_coords : ",  geom_coords)
            geom_coords.append(geom_coords[0])
            geom = Polygon([pix2coord(xypixels, bbox, IMG_HEIGHT, IMG_WIDTH) for xypixels in geom_coords])
    except:
        traceback.print_exc()
        geom = geom
    return geom

IMG_HEIGHT =1024
IMG_WIDTH = 1024 

for mask_img in mask.masks:
    output_mask_np = mask_img.detach().cpu().numpy()
    output_mask_np = cv2.resize(output_mask_np,(IMG_HEIGHT,IMG_WIDTH))
    output_mask_np[np.where(output_mask_np>0)] = 1
    skeleton = pcv.morphology.skeletonize(mask=output_mask_np)
    pruned_skeleton, segmented_image, segment_objects = pcv.morphology.prune(skel_img=skeleton, size=45)
    lines = []
    styles = []
    for line_object in segment_objects:
        line_coords = line_object.squeeze()
        if  len(line_coords.shape) < 2:
            continue
        line_object = LineString(line_coords)
        line_object = convert_pix2coord(line_object, '31313321')
        if line_object.is_valid :
            lines.append(line_object)
            n = 2
            if len(line_coords)>2:
                n = 3
        inference_gdf = gpd.GeoDataFrame({"label" : styles, "tile_id": [quad_key] * len(styles), "task_id": [quad_key] * len(styles)}, geometry=lines)  